# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

import re

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [4]:
path_to_save = './../results/03a_features'

# Loading data

In [5]:
path_to_ftir_data= './../data/processed_data/ftir_extracted_features.csv'
path_to_samples_description = './../data/parsed_data/samples_description.csv'

## FTIR-features

In [6]:
initial_features_df = pd.read_csv(path_to_ftir_data, sep=';', index_col=0)

print(initial_features_df.shape)
initial_features_df.head(3)

(75, 32)


,$\overline{I}_{3800\text{–}2500}$,$I_{3375}$,$\overline{I}_{3375 \pm 32}$,$I_{3293}$,$\overline{I}_{3293 \pm 32}$,$I_{2928}$,$\overline{I}_{2928 \pm 32}$,$\overline{I}_{1800\text{–}900}$,$\overline{I}_{1800\text{–}1500}$,$I_{1722}$,$\overline{I}_{1722 \pm 32}$,$I_{1626}$,$\overline{I}_{1626 \pm 32}$,$I_{1550}$,$\overline{I}_{1550 \pm 32}$,$\overline{I}_{1500\text{–}1300}$,$I_{1396}$,$\overline{I}_{1396 \pm 32}$,$\overline{I}_{1300\text{–}900}$,$I_{1240}$,$\overline{I}_{1240 \pm 32}$,$I_{1155}$,$\overline{I}_{1155 \pm 32}$,$I_{1074}$,$\overline{I}_{1074 \pm 32}$,$I_{877}$,$\overline{I}_{877 \pm 32}$,"$\Sigma I_{p,\ 3800\text{–}2500}$","$\Sigma I_{p,\ 1800\text{–}900}$","$\Sigma I_{p,\ 1800\text{–}1500}$","$\Sigma I_{p,\ 1500\text{–}1300}$","$\Sigma I_{p,\ 1300\text{–}900}$"
division_1_size_bulk,0.526201,0.981207,0.975980,1.000000,0.963685,0.491720,0.492079,0.504366,0.537705,0.426530,0.422144,1.0,0.925096,0.421988,0.453781,0.571735,0.760582,0.726142,0.445780,0.452765,0.443662,0.472792,0.479528,0.681294,0.665771,0.048779,0.049258,2.472926,4.215950,1.848518,0.760582,1.606851
division_1_size_5,0.477909,1.000000,0.995136,0.925090,0.923226,0.406937,0.401135,0.451487,0.497036,0.387542,0.383206,1.0,0.922786,0.357085,0.390761,0.502368,0.687458,0.656020,0.391974,0.426756,0.423141,0.497397,0.498701,0.606138,0.536587,0.093475,0.095869,2.332027,3.962376,1.744626,0.687458,1.530291
division_1_size_3,0.535789,1.000000,0.985816,0.937538,0.935769,0.618445,0.598930,0.517003,0.549005,0.375077,0.381174,1.0,0.938225,0.522055,0.542602,0.523513,0.696553,0.668677,0.489773,0.400246,0.402681,0.683292,0.676252,0.836176,0.731684,0.117973,0.120394,2.555983,4.513399,1.897132,0.696553,1.919714


## Samples description

In [7]:
samples_description = pd.read_csv(f'{path_to_samples_description}', sep=';', index_col=0)

print(samples_description.shape)
samples_description.head(3)

(75, 6)


,Row_ID,Division,Size,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,1,1,bulk,$> 0$,$d > 5$,2
division_1_size_5,2,1,5,$< 5$,$2 < d \leq 5$,1
division_1_size_3,3,1,3,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Function for processing

In [8]:
def generate_transformed_features(data, feature_list, tolerance=50, zero_substitute=1e-8):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    def extract_center(name):
        nums = re.findall(r'\d+', name)
        return int(nums[0]) if nums else None

    def get_interval_tag(center):
        if center is None:
            return None
        if 2500 <= center <= 3800:
            return "high"
        elif 800 <= center <= 1900:
            return "low"
        return None

    def extract_center_and_type(name):
        center = extract_center(name)
        is_avg = r'\overline{I}' in name or '±' in name
        return center, is_avg

    def is_near_duplicate(f1, f2):
        c1, avg1 = extract_center_and_type(f1)
        c2, avg2 = extract_center_and_type(f2)
        if c1 is None or c2 is None:
            return False
        return abs(c1 - c2) <= tolerance and (avg1 or avg2)

    np.seterr(divide='ignore', invalid='ignore')

    for col in feature_list:
        base = data[col].copy()
        base_safe = base.replace(0, zero_substitute)

        data_transformed[wrap_log(col)] = np.log(base_safe)

        for power in [-1]:
            transformed = np.power(base_safe, power)
            transformed.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_transformed[wrap_pow(col, power)] = transformed

    for f1, f2 in combinations(feature_list, 2):
        if is_near_duplicate(f1, f2):
            continue
        c1 = extract_center(f1)
        c2 = extract_center(f2)
        tag1 = get_interval_tag(c1)
        tag2 = get_interval_tag(c2)
        if tag1 != tag2:
            continue

        numerator = data[f1].replace(0, zero_substitute)
        denominator = data[f2].replace(0, zero_substitute)
        ratio = numerator / denominator
        ratio.replace([np.inf, -np.inf], np.nan, inplace=True)

        data_transformed[wrap_div(f1, f2)] = ratio

    return data_transformed

## Calculation process

In [9]:
engineered_features_df = generate_transformed_features(data=initial_features_df, feature_list=initial_features_df.columns, tolerance=50)

print(engineered_features_df.shape)
engineered_features_df.head(3)

(75, 346)


,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,$\log I_{3375}$,$(I_{3375})^{-1}$,$\log \overline{I}_{3375 \pm 32}$,$(\overline{I}_{3375 \pm 32})^{-1}$,$\log I_{3293}$,$(I_{3293})^{-1}$,$\log \overline{I}_{3293 \pm 32}$,$(\overline{I}_{3293 \pm 32})^{-1}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log \overline{I}_{2928 \pm 32}$,$(\overline{I}_{2928 \pm 32})^{-1}$,$\log \overline{I}_{1800\text{–}900}$,$(\overline{I}_{1800\text{–}900})^{-1}$,$\log \overline{I}_{1800\text{–}1500}$,$(\overline{I}_{1800\text{–}1500})^{-1}$,$\log I_{1722}$,$(I_{1722})^{-1}$,$\log \overline{I}_{1722 \pm 32}$,$(\overline{I}_{1722 \pm 32})^{-1}$,$\log I_{1626}$,$(I_{1626})^{-1}$,$\log \overline{I}_{1626 \pm 32}$,$(\overline{I}_{1626 \pm 32})^{-1}$,$\log I_{1550}$,$(I_{1550})^{-1}$,$\log \overline{I}_{1550 \pm 32}$,$(\overline{I}_{1550 \pm 32})^{-1}$,$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,$\log I_{1396}$,$(I_{1396})^{-1}$,$\log \overline{I}_{1396 \pm 32}$,$(\overline{I}_{1396 \pm 32})^{-1}$,$\log \overline{I}_{1300\text{–}900}$,$(\overline{I}_{1300\text{–}900})^{-1}$,$\log I_{1240}$,$(I_{1240})^{-1}$,$\log \overline{I}_{1240 \pm 32}$,$(\overline{I}_{1240 \pm 32})^{-1}$,$\log I_{1155}$,$(I_{1155})^{-1}$,$\log \overline{I}_{1155 \pm 32}$,$(\overline{I}_{1155 \pm 32})^{-1}$,$\log I_{1074}$,$(I_{1074})^{-1}$,$\log \overline{I}_{1074 \pm 32}$,$(\overline{I}_{1074 \pm 32})^{-1}$,...,$\frac{\overline{I}_{1240 \pm 32}}{I_{1074}}$,$\frac{\overline{I}_{1240 \pm 32}}{\overline{I}_{1074 \pm 32}}$,$\frac{\overline{I}_{1240 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1240 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1155}}{I_{1074}}$,$\frac{I_{1155}}{\overline{I}_{1074 \pm 32}}$,$\frac{I_{1155}}{I_{877}}$,$\frac{I_{1155}}{\overline{I}_{877 \pm 32}}$,"$\frac{I_{1155}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1155 \pm 32}}{I_{1074}}$,$\frac{\overline{I}_{1155 \pm 32}}{\overline{I}_{1074 \pm 32}}$,$\frac{\overline{I}_{1155 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1155 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1074}}{I_{877}}$,$\frac{I_{1074}}{\overline{I}_{877 \pm 32}}$,"$\frac{I_{1074}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1074 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1074 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma 

*NaN-values analysis*

In [10]:
nan_df = engineered_features_df.isna().sum()
nan_summary = nan_df[nan_df > 0]
print(nan_summary)

Series([], dtype: int64)


## Merging process

In [11]:
features_df = pd.merge(
    left=initial_features_df,
    right=engineered_features_df,
    how='inner',
    left_index=True,
    right_index=True
)

print(features_df.shape)
features_df.head(3)

(75, 378)


,$\overline{I}_{3800\text{–}2500}$,$I_{3375}$,$\overline{I}_{3375 \pm 32}$,$I_{3293}$,$\overline{I}_{3293 \pm 32}$,$I_{2928}$,$\overline{I}_{2928 \pm 32}$,$\overline{I}_{1800\text{–}900}$,$\overline{I}_{1800\text{–}1500}$,$I_{1722}$,$\overline{I}_{1722 \pm 32}$,$I_{1626}$,$\overline{I}_{1626 \pm 32}$,$I_{1550}$,$\overline{I}_{1550 \pm 32}$,$\overline{I}_{1500\text{–}1300}$,$I_{1396}$,$\overline{I}_{1396 \pm 32}$,$\overline{I}_{1300\text{–}900}$,$I_{1240}$,$\overline{I}_{1240 \pm 32}$,$I_{1155}$,$\overline{I}_{1155 \pm 32}$,$I_{1074}$,$\overline{I}_{1074 \pm 32}$,$I_{877}$,$\overline{I}_{877 \pm 32}$,"$\Sigma I_{p,\ 3800\text{–}2500}$","$\Sigma I_{p,\ 1800\text{–}900}$","$\Sigma I_{p,\ 1800\text{–}1500}$","$\Sigma I_{p,\ 1500\text{–}1300}$","$\Sigma I_{p,\ 1300\text{–}900}$",$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,$\log I_{3375}$,$(I_{3375})^{-1}$,$\log \overline{I}_{3375 \pm 32}$,$(\overline{I}_{3375 \pm 32})^{-1}$,$\log I_{3293}$,$(I_{3293})^{-1}$,$\log \overline{I}_{3293 \pm 32}$,$(\overline{I}_{3293 \pm 32})^{-1}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log \overline{I}_{2928 \pm 32}$,$(\overline{I}_{2928 \pm 32})^{-1}$,$\log \overline{I}_{1800\text{–}900}$,$(\overline{I}_{1800\text{–}900})^{-1}$,$\log \overline{I}_{1800\text{–}1500}$,$(\overline{I}_{1800\text{–}1500})^{-1}$,...,$\frac{\overline{I}_{1240 \pm 32}}{I_{1074}}$,$\frac{\overline{I}_{1240 \pm 32}}{\overline{I}_{1074 \pm 32}}$,$\frac{\overline{I}_{1240 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1240 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1240 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1155}}{I_{1074}}$,$\frac{I_{1155}}{\overline{I}_{1074 \pm 32}}$,$\frac{I_{1155}}{I_{877}}$,$\frac{I_{1155}}{\overline{I}_{877 \pm 32}}$,"$\frac{I_{1155}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1155}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1155 \pm 32}}{I_{1074}}$,$\frac{\overline{I}_{1155 \pm 32}}{\overline{I}_{1074 \pm 32}}$,$\frac{\overline{I}_{1155 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1155 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1155 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1074}}{I_{877}}$,$\frac{I_{1074}}{\overline{I}_{877 \pm 32}}$,"$\frac{I_{1074}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1074}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1074 \pm 32}}{I_{877}}$,$\frac{\overline{I}_{1074 \pm 32}}{\overline{I}_{877 \pm 32}}$,"$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1074 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{877}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{877 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1500\text{–}1300}}$"

# Creating features description

## Functions for processing

In [12]:
def classify_feature_type(base_columns, columns):
    base_columns = [col for col in base_columns if "Unnamed" not in col]
    columns = [col for col in columns if "Unnamed" not in col]

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def is_interval(col):
        col = clean(col)
        return r"\overline{I}_{" in col or r"\Sigma I_{p," in col

    def is_peak(col):
        return not is_interval(col)

    def extract_numbers(col):
        return list(map(int, re.findall(r"\d{3,4}", col)))

    def is_in_range(col, low, high):
        return any(low <= num <= high for num in extract_numbers(clean(col)))

    def classify(col, base=True):
        col_clean = clean(col)
        source = "intervals" if is_interval(col_clean) else "peaks"

        if is_in_range(col_clean, 2500, 3800):
            region = "3800_2500"
        elif is_in_range(col_clean, 800, 1900):
            region = "1900_800"
        else:
            region = "unknown"

        kind = "base" if base else "engineered"
        return f"{kind}_features_{source}_{region}"

    groups = {}

    for col in base_columns:
        key = classify(col, base=True)
        groups.setdefault(key, []).append(col)

    for col in columns:
        if col not in base_columns:
            key = classify(col, base=False)
            groups.setdefault(key, []).append(col)

    groups["base_features"] = base_columns
    groups["engineered_features"] = [col for col in columns if col not in base_columns]

    return groups

## Feature groups selection

In [13]:
base_columns = initial_features_df.columns.tolist()
columns = features_df.columns.tolist()

In [14]:
features_description_dict = classify_feature_type(base_columns=base_columns, columns=columns)

In [15]:
for feature_type, feature_lst in features_description_dict.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_intervals_3800_2500: 5
(*) base_features_peaks_3800_2500: 3
(*) base_features_intervals_1900_800: 16
(*) base_features_peaks_1900_800: 8
(*) engineered_features_intervals_3800_2500: 31
(*) engineered_features_peaks_3800_2500: 9
(*) engineered_features_intervals_1900_800: 262
(*) engineered_features_peaks_1900_800: 44
(*) base_features: 32
(*) engineered_features: 346


# Saving data

In [16]:
features_df.to_csv('./../data/processed_data/ftir_features_long_list.csv', sep=';')

In [17]:
with open('./../data/processed_data/features_description.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict, f, ensure_ascii=False, indent=2)